<a href="https://colab.research.google.com/github/saulantonine/Redes-Neuronales/blob/main/Color_predominante_en_cubo_Rubik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import layers, models


In [3]:

# Estableciendo valores numericos a las cadenas
COLOR = {
    "r": 0,
    "v": 1,
    "z": 2,
    "a": 3,
    "n": 4,
    "b": 5,
    "none": 6  # para empates se usa "none"
}


In [4]:
# Diccionario que traduce numeros a letras
NUM_COLOR = {v: k for k, v in COLOR.items()}

In [5]:

# Manipulacion de set de datos
def set_datos(archivo, carpeta):
    with open(archivo, "r") as f:
        contenido = json.load(f)

    imagenes = []
    etiquetas = []

    for nombre_img, color_predom in contenido.items():
        # Cargar y normalizar
        dir_img = os.path.join(carpeta, nombre_img)
        imagen = tf.keras.utils.load_img(dir_img, target_size=(224, 224))
        imagen = tf.keras.utils.img_to_array(imagen) / 255.0  # Normalizar a [0, 1]

        # De color a numero con el caso en el que no halla color predominante
        if color_predom == 0:
            etiqueta_num = COLOR["none"]
        else:
            etiqueta_num = COLOR[color_predom]

        imagenes.append(imagen)
        etiquetas.append(etiqueta_num)

    return np.array(imagenes), np.array(etiquetas)

In [6]:

# modelo CNN
def crear_modelCNN():
    modeloCNN = models.Sequential([
        layers.Input(shape=(224, 224, 3)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(7, activation='softmax')  # 7 classes (6 colors + none)
    ])
    return modeloCNN

In [7]:


# nombres de archivo y carpeta
archivo = "etiquetas.json"
carpeta = "Imagenes"


imagenes, etiquetas = set_datos(archivo, carpeta)

X_train, X_val, y_train, y_val = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

modelo = crear_modelCNN()
modelo.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

modelo.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
)

perdida, precision = modelo.evaluate(X_val, y_val)
print(f"Precision de validacion: {precision:.2f}")

# Guardar
modelo.save("cubo_rubik_color_predominante.h5")

# Evaluar el modelo con imagenes nuevas
predicciones = modelo.predict(X_val[:5])  # Usando 5 imagenes de validacion
color_predecido = np.argmax(predicciones, axis=1)

print("Predicciones:")
for i, pred in enumerate(color_predecido):
    print(f"Imagen {i + 1}: Color: {NUM_COLOR[pred]}, Color Real: {NUM_COLOR[y_val[i]]}")


FileNotFoundError: [Errno 2] No such file or directory: 'etiquetas.json'